# Model assembly

Wir entschieden uns ein Neuronales Netz zu benutzen, da es uns ermöglicht durch Machinelles Lernen eine automatische Gewichtung der verschiedenen Eingangsstatistiken zu bestimmen.

## Modellimplementierung

### Imports
Importiert die Bibliotheken PyTorch und Pandas, sowie Matplotlib um Lernkurven graphisch darzustellen.

In [ ]:
import time
import pandas as pd
import torch
from torch import nn
from torch.utils.data import Dataset
from torch.utils.data import DataLoader
from torch.utils.data import random_split
import matplotlib.pyplot as plt

###### Device
Bestimmt ob auf der Grafikkarte oder auf dem Prozessor gerechnet werden soll.
Wenn eine CUDA fähige Grafikkarte erkannt wird, wird diese als Rechengerät ausgewählt.
 Dies spart Rechenzeit, da CUDA Kerne deutlich effizienter in der Berechnung Neuronaler Netze sind.

In [ ]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'
print('Using {} device'.format(device))

### PyTorch Klassen
Definiert eine Klasse für das Datenset und eine für das Neuronale Netzwerk.

In [ ]:
class LolProDataset(Dataset):
    def __init__(self, data_file):
        # Preload data into tensors
        data = pd.read_csv(data_file)
        self.labels = torch.tensor(data.pop('Win Rate').to_numpy(), device=device)
        self.features = torch.tensor(data.to_numpy(), device=device)

    def __len__(self):
        # Number of rows in the dataset
        return self.features.shape[0]

    def __getitem__(self, idx):
        # Returns item (features, label) at specific index
        x = self.features[idx]
        y = self.labels[idx]
        return (x, y)

    def split(self, test_rate):
        # Returns number of items for test and train sets by given test_rate
        testc = int(self.__len__()*test_rate)
        trainc = int(self.__len__() - testc)
        return [trainc, testc]

# Implementierung des neuronalen Netzes auf Basis der Vererbung vom nn.Modul
class LolProNetwork(nn.Module):
    def __init__(self, network_stack):
        super(LolProNetwork, self).__init__()
        self.network_stack = network_stack

    def forward(self, x):
        return self.network_stack(x)

###### Train and Test Loops
Loop in dem das Neuronale Netz trainiert und getestet wird.
Der mittlere Fehler aus jedem Batch wird genutzt um die Gewichte im Netzwerk im nächsten Batch anzupassen.
Mit Backpropagation werden die Errors durch das Netzwerk zurückgeführt und die Gewichte entsprechend angepasst werden.

In [ ]:
def train_loop(dataloader, model, loss_fn, optimizer):
    # Während der Trainingszeit ausgeführt
    size = len(dataloader.dataset)
    for batch, (X, y) in enumerate(dataloader):
        # Berechnung des Loss durch die Prediction
        pred = model(X)
        loss = loss_fn(pred, y.unsqueeze(1))

        # Backpropagation
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

def test_loop(dataloader, model, loss_fn, optimizer):
    # Wird auf dem trainierten Netzwerk während der Testzeit ausgeführt
    size = len(dataloader.dataset)
    test_loss, err = 0, 0

    with torch.no_grad():
        for X, y in dataloader:
            pred = model(X)
            test_loss += loss_fn(pred, y.unsqueeze(1)).item()
            err += torch.abs(pred - y.unsqueeze(1)).sum().data
    test_loss /= size
    err /= size
    return (err.item(), test_loss)

###### Run Code
Set parameters, define Network, loss function and optimizer, load in Dataset and train the network
The Learning Rate is

Die Epochen beschreiben wie oft das training mit anderer batch Reihenfolge durchgeführt werden soll, damit keine Biase
zum Ende des Trainingssatzes entstehen. Also wie oft die Trainfunktion auf das Netz ausgeführt werden soll

Die Größe der der Batches entscheidet wie viele Zeilen für jede Neuberechnung genutzt werden sollen. Größere Batches können zu Underfitting und kleinere zu Overfitting führen
Die Learning Rate ist der Faktor der bestimmt wie weit die Weights in jedem Durchlauf an das vorherige Ergebnis angepasst werden sollen.

Der `network_stack` beschreibt den Aufbau des Netzwerkes.
Der erste Netzwerklayer wird mit den 20 Datenpunkten gefüttert.
Die Aktivierungsfunktion für den Hiddenlayer ist die ReLu Funktion
![img](https://pytorch.org/docs/stable/_images/ReLU.png)
Die Ausgabe wird mit einer Sigmoid Funktion aktiviert.

In [ ]:
lr = 0.0001 # Learning Rate
batch_size = 20 # Batch Größe (Parallel berechnete Zeilen)
epochs = 10000 # Epochen (Iterationen)

# Network
network_stack = nn.Sequential(
    nn.Dropout(p=0.2),
    nn.Sigmoid(),
    nn.Linear(19, 8),
    nn.PReLU(),
    nn.Linear(8, 1),
    nn.Softsign()
)

# Erstelle das Model auf basis des Netzwerks und lasse es auf dem Gerät (cpu oder cuda) berechnen
model = LolProNetwork(network_stack).to(device).double()

loss_fn = nn.L1Loss() # Mean average Loss function
optimizer = torch.optim.Adam(model.parameters(), lr) # Optimizer

# Load Dataset
dataset = LolProDataset('cleanData.csv')
train_data, test_data = random_split(dataset, dataset.split(0.1), generator=torch.Generator().manual_seed(42))
train_dataloader = DataLoader(train_data, batch_size=batch_size, shuffle=True)
test_dataloader = DataLoader(test_data, batch_size=batch_size, shuffle=True)

# Training in jeder Epoche
history = pd.DataFrame([], columns=["Epoch", "MAE", "Loss"])
for t in range(epochs):
    train_loop(train_dataloader, model, loss_fn, optimizer)
    res = test_loop(test_dataloader, model, loss_fn, optimizer)
    history = history.append({"Epoch": t+1, "MAE": res[0], "Loss": res[1]}, ignore_index=True)
    if (t+1)%100 == 0:
        print(f"Epoch {t+1} - MAE: {res[0]}, Loss: {res[1]}")

### Visualize results
Mithilfe von Matplotlib werden die Ergebnisse graphisch dargestellt.

In [ ]:
def plotit(history):
    fig, axs = plt.subplots(1, 2)

    axs[0].plot(history['Epoch'], history['MAE'])
    axs[0].set_xlabel("Epochs")
    axs[0].set_ylabel("MAE")
    axs[1].plot(history['Epoch'], history['Loss'])
    axs[1].set_xlabel("Epochs")
    axs[1].set_ylabel("Loss")

    fig.tight_layout()

    plt.show()
plotit(history)

Diese Ergebnisse sehen schon ganz gut aus. Es ist eine deutliche Lernkurve zu erkennen und ein Ergebnis von ca. 89 % Genauigkeit ist auch in Ordnung. Das können wir jedoch noch verbessern!

## Grid Search

Um eine bessere Architektur und bessere Parameter für das Training zu finden benutzen wir eine Grid-Suche, welche verschiedene Kombinationen aus-testet und die Ergebnisse für spätere Analyse speichert. Hierzu implementieren wir eine Trainer Funktion, welche die verschiedenen Netzwerke und Hyperparameter als Parameter entgegen nimmt und diese austestet.

In [ ]:
def trainer(lr, batch_size, epochs, network, opt, dataset):
    model = LolProNetwork(network).to(device).double()
    loss_fn = nn.L1Loss() # MAE Loss function
    optimizer = opt(model.parameters(), lr) # Optimizer

    dataset = LolProDataset(dataset)
    train_data, test_data = random_split(dataset, dataset.split(0.1), generator=torch.Generator().manual_seed(42))
    train_dataloader = DataLoader(train_data, batch_size=batch_size, shuffle=True)
    test_dataloader = DataLoader(test_data, batch_size=batch_size, shuffle=True)
    history = pd.DataFrame([], columns=["Epoch", "MAE", "Loss"])

    print("\n-----------------------------------")
    print(f"\nStart Training with the Parameters:\nlearning rate: {lr}\nbatch size: {batch_size}\nepochs: {epochs}\noptimizer: {optimizer}\nnetwork: {network}\n")
    for t in range(epochs):
        train_loop(train_dataloader, model, loss_fn, optimizer)
        res = test_loop(test_dataloader, model, loss_fn, optimizer)
        history = history.append({"Epoch": t+1, "MAE": res[0], "Loss": res[1]}, ignore_index=True)
    
    print(f"\nSmallest MAE at index {history['MAE'].idxmin()} with:\n{history['MAE'].min()}\n")
    plotit(history)
    print("-----------------------------------\n")
    return (model, history)


Nun iterieren wir über verschiedene Parameter um die verschiedenen Kombinationen mit der Trainer Funktion testen zu lassen. Dies dauert eine Weile, da jedes Training seine Zeit braucht.

In [ ]:
learning_rates = [0.005, 0.0005, 0.00005]
batch_sizes = [15, 45, 90]
e = 1000
optimizer = [torch.optim.Adam, torch.optim.SGD]
networks = [
    nn.Sequential(
        nn.Linear(19, 5),
        nn.CELU(),
        nn.Linear(5, 1),
        nn.Sigmoid()
    ),
    nn.Sequential(
        nn.Linear(19, 64),
        nn.ReLU(),
        nn.Linear(64, 1),
        nn.Sigmoid()
    )
]

results = pd.DataFrame([], columns=["Learning Rate", "Batch Size", "Epochs", "Optimizer", "Network", "History", "Min MAE", "Idx Min MAE", "Duration"])
 
for lr in learning_rates:
    for bs in batch_sizes:
        if bs == 1 or bs == 10:
            continue
        for opt in optimizer:
            for net in networks:
                stime = time.time()
                mod, res = trainer(lr, bs, e, net, opt, 'cleanData.csv')
                etime = time.time()
                results = results.append({
                    "Learning Rate": lr,
                    "Batch Size": bs,
                    "Epochs": e,
                    "Optimizer": opt,
                    "Network": net,
                    "History": res,
                    "Min MAE": res['MAE'].min(),
                    "Idx Min MAE": res['MAE'].idxmin(),
                    "Duration": (etime - stime) * 1000
                }, ignore_index=True)
results.to_pickle('grid_search_results')
results

Mithilfe von Pandas können wir uns das beste Ergebnis ausgeben lassen.

In [ ]:
plotit(results.iloc[results['Min MAE'].idxmin()]['History'])
results['Min MAE'].min()

Ein Blick über andere Kombination verrät jedoch, dass andere Kombinationen eventuell viel versprechender sein können als diese, wenn wir sie ein wenig länger Trainieren lassen.

In [ ]:
lr = 0.00005 # Learning Rate
batch_size = 45 # Batch Size (Parallel calculated rows)
epochs = 3000 # Epochs (iterations over dataset)

# Network
network_stack = nn.Sequential(
    nn.Linear(19, 5),
    nn.CELU(),
    nn.Linear(5, 1),
    nn.Sigmoid()
)

model, history = trainer(lr, batch_size, epochs, network_stack, torch.optim.Adam, 'cleanData.csv')

Das sieht schon besser aus. Durch mehr Epochen konnte das Netzwerk immer besser werden. Der Trainingseffekt nimmt jedoch nach einiger Zeit ab und es besteht die Gefahr eines Overfittings. Daher sind 3000 Epochen perfekt für unsere Zwecke.

## Beispiel

In [ ]:
idx = 0
data = pd.read_csv('cleanData.csv')
rates = data.pop('Win Rate')
t1 = torch.tensor(data.iloc[idx].to_numpy(), device=device).double().unsqueeze(0)
t1

In [ ]:
rates.iloc[idx]

In [ ]:
model(t1).item()

## Modell speichern

Um das Modell in einem Webserver oder ähnlichem wieder zu verwenden wird es gespeichert.

In [ ]:
torch.save(model.state_dict(), './model/lol_predicter_v1')